In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import os
import time

# Required by SEC
header = {'User-Agent': "db101@rice.edu"}

# Get tickers for companies
tickers_cik = requests.get('https://www.sec.gov/files/company_tickers.json', headers=header)
tickers_cik = pd.json_normalize(pd.json_normalize(tickers_cik.json(), max_level=0).values[0])
tickers_cik['cik_str'] = tickers_cik['cik_str'].astype(str).str.zfill(10)

display(tickers_cik.head())

,cik_str,ticker,title
0,0000789019,MSFT,MICROSOFT CORP
1,0000320193,AAPL,Apple Inc.
2,0001045810,NVDA,NVIDIA CORP
3,0001652044,GOOGL,Alphabet Inc.
4,0001018724,AMZN,AMAZON COM INC


In [2]:
# List of companies we are interested in
tickers = ['BP', 'COP', 'CVX', 'CXO', 'DVN', 'EOG', 'EQNR', 'HES',
           'MPC', 'MRO', 'OXY', 'PDCE', 'PSX', 'PXD', 'SHEL', 'TTE',
           'VLO', 'XOM']

# Filter CIK to tickers
filtered_tickers_cik = tickers_cik[tickers_cik['ticker'].isin(tickers)]

# Identify missing tickers
missing_tickers = [ticker for ticker in tickers if ticker not in filtered_tickers_cik['ticker'].values]

# Display the filtered dataframe and the list of missing tickers
display(filtered_tickers_cik)

print("\nMissing Tickers:")
print(missing_tickers)

,cik_str,ticker,title
15,0000034088,XOM,EXXON MOBIL CORP
29,0000093410,CVX,CHEVRON CORP
40,0001306965,SHEL,Shell plc
66,0000879764,TTE,TotalEnergies SE
85,0001163165,COP,CONOCOPHILLIPS
123,0000313807,BP,BP PLC
154,0001140625,EQNR,EQUINOR ASA
185,0000821189,EOG,EOG RESOURCES INC
214,0001510295,MPC,Marathon Petroleum Corp
222,0001534701,PSX,Phillips 66



Missing Tickers:
['CXO', 'PDCE', 'PXD']


In [3]:
# Show the data structure for the first company in the list, Exxon
# Get the cik and put it into the URL
test_cik = filtered_tickers_cik.iloc[0, 0]
test_url = f"https://data.sec.gov/submissions/CIK{str(test_cik)}.json"
print(test_url)

# Get a list of the 1000 most recent company filings
company_filings = requests.get(test_url, headers=header).json()
company_filings_df = pd.DataFrame(company_filings["filings"]["recent"])

# Show only the 10-k's
display(company_filings_df[company_filings_df.form == "10-K"])

https://data.sec.gov/submissions/CIK0000034088.json


,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
36,0000034088-24-000018,2024-02-28,2023-12-31,2024-02-28T16:37:18.000Z,34,10-K,001-02256,24696144,,25901276,1,1,xom-20231231.htm,FORM 10-K
188,0000034088-23-000020,2023-02-22,2022-12-31,2023-02-22T16:37:14.000Z,34,10-K,001-02256,23653785,,27495176,1,1,xom-20221231.htm,FORM 10-K
286,0000034088-22-000011,2022-02-23,2021-12-31,2022-02-23T16:11:16.000Z,34,10-K,001-02256,22663708,,23981301,1,1,xom-20211231.htm,FORM 10-K
627,0000034088-21-000012,2021-02-24,2020-12-31,2021-02-24T17:01:02.000Z,34,10-K,001-02256,21674541,,21572229,1,1,xom-20201231.htm,FORM 10-K
784,0000034088-20-000016,2020-02-26,2019-12-31,2020-02-26T16:15:19.000Z,34,10-K,001-02256,20655849,,31993281,1,1,xom10k2019.htm,FORM 10-K
877,0000034088-19-000010,2019-02-27,2018-12-31,2019-02-27T16:19:37.000Z,34,10-K,001-02256,19637956,,24066786,1,0,xom10k2018.htm,FORM 10-K
977,0000034088-18-000015,2018-02-28,2017-12-31,2018-02-28T16:39:51.000Z,34,10-K,001-02256,18651779,,23712262,1,0,xom10k2017.htm,FORM 10-K


In [4]:
# Additional files if you need to go back further in time
additional_files = company_filings.get("filings", {}).get("files", [])
additional_files_df = pd.DataFrame(additional_files)
display(additional_files_df)

file_link = additional_files_df.iloc[0, 0]
old_files_test_url = f"https://data.sec.gov/submissions/{str(file_link)}"
print(old_files_test_url)

# Get a list of the first set of older filinngs
older_filings = requests.get(old_files_test_url, headers=header).json()
older_filings_df = pd.DataFrame(older_filings)

# Show only the 10-k's
display(older_filings_df[older_filings_df.form == "10-K"])

,name,filingCount,filingFrom,filingTo
0,CIK0000034088-submissions-001.json,2000,2004-06-25,2018-01-02
1,CIK0000034088-submissions-002.json,359,1994-03-04,2004-06-24


https://data.sec.gov/submissions/CIK0000034088-submissions-001.json


,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
97,0000034088-17-000017,2017-02-22,2016-12-31,2017-02-22T16:47:56.000Z,34,10-K,001-02256,17629081,,23416048,1,0,xom10k2016.htm,FORM 10-K
229,0000034088-16-000065,2016-02-24,2015-12-31,2016-02-24T16:09:22.000Z,34,10-K,001-02256,161452039,,20962361,1,0,xom10k2015.htm,FORM 10-K
355,0000034088-15-000013,2015-02-25,2014-12-31,2015-02-25T16:26:23.000Z,34,10-K,001-02256,15648169,,26917781,1,0,xom10k2014.htm,FORM 10-K
476,0000034088-14-000012,2014-02-26,2013-12-31,2014-02-26T16:07:02.000Z,34,10-K,001-02256,14644672,,27141736,1,0,xom10k2013.htm,FORM 10-K
596,0000034088-13-000011,2013-02-27,2012-12-31,2013-02-27T16:05:06.000Z,34,10-K,001-02256,13646760,,27580538,1,0,xom10k2012.htm,FORM 10-K
726,0001193125-12-078102,2012-02-24,2011-12-31,2012-02-24T16:08:32.000Z,34,10-K,001-02256,12638257,,25918019,1,0,d257530d10k.htm,FORM 10-K
880,0001193125-11-047394,2011-02-25,2010-12-31,2011-02-25T16:07:35.000Z,34,10-K,001-02256,11640992,,31425179,1,0,d10k.htm,FORM 10-K
1088,0001193125-10-042929,2010-02-26,2009-12-31,2010-02-26T16:17:17.000Z,34,10-K,001-02256,10639543,,10097184,1,0,d10k.htm,FORM 10-K
1249,0001193125-09-040966,2009-02-27,2008-12-31,2009-02-27T16:16:54.000Z,34,10-K,001-02256,09643163,,3026718,0,0,d10k.htm,FORM 10-K
1401,0001193125-08-041781,2008-02-28,2007-12-31,2008-02-28T16:17:53.000Z,34,10-K,001-02256,08650851,,2925185,0,0,d10k.htm,FORM 10-K


In [5]:
# Get the accession number and file_name of the most recent 10-k filing and put it into a URL
access_number = company_filings_df[company_filings_df.form == "10-K"].accessionNumber.values[0].replace("-", "")
file_name = company_filings_df[company_filings_df.form == "10-K"].primaryDocument.values[0]
tenk_test_url = f"https://www.sec.gov/Archives/edgar/data/{test_cik}/{access_number}/{file_name}"
print(tenk_test_url)
print()

# Convert to text in the html to a string
test_response = requests.get(tenk_test_url, headers=header)
test_html_content = test_response.content

# Parse and extract the HTML content using BeautifulSoup
soup = BeautifulSoup(test_html_content, 'html.parser')
test_string = soup.get_text(separator="\n", strip=True)
print(test_string)

https://www.sec.gov/Archives/edgar/data/0000034088/000003408824000018/xom-20231231.htm

xom-20231231
FALSE
FY
0000034088
http://www.exxonmobil.com/20231231#CrudeOilAndProductPurchases
http://fasb.org/us-gaap/2023#DepreciationDepletionAndAmortization http://fasb.org/us-gaap/2023#ExplorationExpense
http://fasb.org/us-gaap/2023#OtherAssetsNoncurrent
http://fasb.org/us-gaap/2023#OtherAssetsNoncurrent
http://fasb.org/us-gaap/2023#PropertyPlantAndEquipmentNet
http://fasb.org/us-gaap/2023#PropertyPlantAndEquipmentNet
http://fasb.org/us-gaap/2023#AccountsPayableAndAccruedLiabilitiesCurrent
http://fasb.org/us-gaap/2023#AccountsPayableAndAccruedLiabilitiesCurrent
http://fasb.org/us-gaap/2023#OtherLiabilitiesNoncurrent
http://fasb.org/us-gaap/2023#OtherLiabilitiesNoncurrent
http://fasb.org/us-gaap/2023#OtherAssetsNoncurrent http://fasb.org/us-gaap/2023#ReceivablesNetCurrent
http://fasb.org/us-gaap/2023#OtherAssetsNoncurrent http://fasb.org/us-gaap/2023#ReceivablesNetCurrent
http://fasb.org/us-gaa

## Do not run the cell below unless you want to redownload!!!

It just initiates the csv file, if you uncomment and run it will delete the existing file!!!

In [6]:
# # This cell just initiates the download log with the correct headers
# # It only needs to be run once!
# # If you run again it will delete the log entries
# log_df = pd.DataFrame(columns=['Ticker', 'Title', 'CIK', 'AccessionNumber', 'Form', 'Date', 'URL'])
# log_df.to_csv('download_log.csv', index=False)

In [7]:
# Function to download forms
def download(cik_list, tickers_cik, header={'User-Agent': "db101@rice.edu"}):
    count = 0
    function_start_time = time.time()

    # Loop through the CIKs
    for cik in cik_list:
        count += 1
        # Find ticker and title for the current CIK
        ticker_info = tickers_cik[tickers_cik['cik_str'] == cik]
        if ticker_info.empty:
            print(f"Ticker and title not found for CIK {cik}")
            continue
        ticker = ticker_info['ticker'].values[0]
        title = ticker_info['title'].values[0]

        # Get the url and check if the response is successful
        url = f"https://data.sec.gov/submissions/CIK{str(cik)}.json"
        response = requests.get(url, headers=header)
        if response.status_code != 200:
            print(f"Failed to retrieve data for {title} (CIK {cik}). Status code: {response.status_code}")
            continue  # Skips to the next loop

        # Get the latest 1000 company filings
        company_filings = response.json()
        company_filings_df = pd.DataFrame(company_filings["filings"]["recent"])
        time.sleep(0.2) # Delay as per SEC guidelines
        
        # Add any additional filings to the df
        additional_files = company_filings.get("filings", {}).get("files", [])
        time.sleep(0.2) # Delay as per SEC guidelines
        
        # Loop through the file names to add older files
        if additional_files:
            additional_files_df = pd.DataFrame(additional_files)

            for file_name in additional_files_df['name']:
                old_files_url = f"https://data.sec.gov/submissions/{str(file_name)}"

                # Get a list of the first set of older filings and concatenate to company filings
                older_filings = requests.get(old_files_url, headers=header).json()
                older_filings_df = pd.DataFrame(older_filings)
                time.sleep(0.2) # Delay as per SEC guidelines
                
                # Concatenate
                company_filings_df = pd.concat([company_filings_df, older_filings_df], ignore_index=True)
        else:
            print("No additional files found.")

        # Filter 10-K, 10-Q, 8-K, and 11-K filings
        forms_to_filter = ["10-K", "10-Q", "8-K", "11-K"]
        filtered_filings = company_filings_df[company_filings_df.form.isin(forms_to_filter)]
        
        # Filter to 2019 and beyond
        company_filings_df['filingDate'] = pd.to_datetime(company_filings_df['filingDate'], errors='coerce')
        filtered_filings = filtered_filings[filtered_filings['filingDate'] >= '2019-01-01']
        
        if filtered_filings.empty:
            print(f"No {', '.join(forms_to_filter)} filings found for {title} (CIK {cik})")
            new_log_row = pd.DataFrame({'Ticker': [ticker], 'Title': [title], 'CIK': [cik], 'AccessionNumber': [f'No {", ".join(forms_to_filter)} Filings Found']})
            new_log_row.to_csv('download_log.csv', mode='a', header=False, index=False)
            continue

        # Loop through all available filings
        for _, filing in filtered_filings.iterrows():
            access_number = filing['accessionNumber'].replace("-", "")
            file_name = filing['primaryDocument']
            url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{access_number}/{file_name}"
            form = filing['form']
            date = filing['filingDate']
            save_directory = os.path.join("..", "SEC_data")
            os.makedirs(save_directory, exist_ok=True)
            save_name = os.path.join(save_directory, f"{access_number}_{ticker}_{form}_{date}.html")

            # Check if the response is successful
            response = requests.get(url, headers=header)
            if response.status_code != 200:
                print(f"Failed to download 10-K for {title} (CIK {cik}, Accession Number: {access_number}). Status code: {response.status_code}")
                continue  # Skip to the next filing

            # Save the document
            if file_name:
                content = response.content.decode("utf-8", errors='ignore')
                with open(save_name, "w", encoding="utf-8") as f:
                    f.write(content)
            else:
                print(f"Primary document name is missing for {title} (CIK {cik}, Accession Number: {access_number}). Skipping download.")
                continue

            # Log the successful download into a csv. file
            new_log_row = pd.DataFrame({'Ticker': [ticker], 'Title': [title], 'CIK': [cik], 'AccessionNumber': [access_number], 'Form': [form], 'Date': [date], 'URL': [url]})
            new_log_row.to_csv('download_log.csv', mode='a', header=False, index=False)
            time.sleep(0.2) # Delay as per SEC guidelines

            # Notification of a successful download
            elapsed_time = time.time() - function_start_time
            elapsed_minutes = elapsed_time / 60.0
            print(f'Done w/ {title} (CIK: {cik}), Accession #: {access_number}. CIK count: {count}. Total Time: {elapsed_minutes:.2f} mins')

    return

In [8]:
# Find the most recent cik number in the log
log_df = pd.read_csv('download_log.csv')
last_cik = str(log_df['CIK'].iloc[-1]) if not log_df.empty else '0000034088'  # Default to Exxon
last_cik_padded = last_cik.zfill(10)

# Find the matching row in the tickers_cik df
filtered_tickers_cik = filtered_tickers_cik.reset_index(drop=True)
last_cik_row = filtered_tickers_cik[filtered_tickers_cik['cik_str'] == last_cik_padded]

# Show results
display(last_cik_row)

,cik_str,ticker,title
0,0000034088,XOM,EXXON MOBIL CORP


In [9]:
# If you just run all the code, the cik_list should automatically pick up where
# the downloads last left off.  It will repeat the last cik just to make sure it gets it all.
cik_list = filtered_tickers_cik['cik_str'].iloc[last_cik_row.index[0]:]

# Execute the function
download(cik_list, filtered_tickers_cik)

Done w/ EXXON MOBIL CORP (CIK: 0000034088), Accession #: 000095010324006564. CIK count: 1. Total Time: 0.07 mins
Done w/ EXXON MOBIL CORP (CIK: 0000034088), Accession #: 000095010324006322. CIK count: 1. Total Time: 0.08 mins
Done w/ EXXON MOBIL CORP (CIK: 0000034088), Accession #: 000003408824000029. CIK count: 1. Total Time: 0.09 mins
Done w/ EXXON MOBIL CORP (CIK: 0000034088), Accession #: 000003408824000025. CIK count: 1. Total Time: 0.11 mins
Done w/ EXXON MOBIL CORP (CIK: 0000034088), Accession #: 000003408824000021. CIK count: 1. Total Time: 0.13 mins
Done w/ EXXON MOBIL CORP (CIK: 0000034088), Accession #: 000003408824000018. CIK count: 1. Total Time: 0.14 mins
Done w/ EXXON MOBIL CORP (CIK: 0000034088), Accession #: 000003408824000015. CIK count: 1. Total Time: 0.16 mins
Done w/ EXXON MOBIL CORP (CIK: 0000034088), Accession #: 000003408824000007. CIK count: 1. Total Time: 0.17 mins
Done w/ EXXON MOBIL CORP (CIK: 0000034088), Accession #: 000003408824000003. CIK count: 1. Total

Done w/ EXXON MOBIL CORP (CIK: 0000034088), Accession #: 000003408821000004. CIK count: 1. Total Time: 1.24 mins
Done w/ EXXON MOBIL CORP (CIK: 0000034088), Accession #: 000003408820000100. CIK count: 1. Total Time: 1.26 mins
Done w/ EXXON MOBIL CORP (CIK: 0000034088), Accession #: 000003408820000095. CIK count: 1. Total Time: 1.27 mins
Done w/ EXXON MOBIL CORP (CIK: 0000034088), Accession #: 000003408820000093. CIK count: 1. Total Time: 1.29 mins
Done w/ EXXON MOBIL CORP (CIK: 0000034088), Accession #: 000003408820000090. CIK count: 1. Total Time: 1.31 mins
Done w/ EXXON MOBIL CORP (CIK: 0000034088), Accession #: 000003408820000087. CIK count: 1. Total Time: 1.32 mins
Done w/ EXXON MOBIL CORP (CIK: 0000034088), Accession #: 000003408820000081. CIK count: 1. Total Time: 1.34 mins
Done w/ EXXON MOBIL CORP (CIK: 0000034088), Accession #: 000003408820000073. CIK count: 1. Total Time: 1.36 mins
Done w/ EXXON MOBIL CORP (CIK: 0000034088), Accession #: 000003408820000066. CIK count: 1. Total

Done w/ CHEVRON CORP (CIK: 0000093410), Accession #: 000009341022000040. CIK count: 2. Total Time: 2.45 mins
Done w/ CHEVRON CORP (CIK: 0000093410), Accession #: 000009341022000036. CIK count: 2. Total Time: 2.46 mins
Done w/ CHEVRON CORP (CIK: 0000093410), Accession #: 000009341022000035. CIK count: 2. Total Time: 2.47 mins
Done w/ CHEVRON CORP (CIK: 0000093410), Accession #: 000009341022000026. CIK count: 2. Total Time: 2.49 mins
Done w/ CHEVRON CORP (CIK: 0000093410), Accession #: 000009341022000023. CIK count: 2. Total Time: 2.50 mins
Done w/ CHEVRON CORP (CIK: 0000093410), Accession #: 000095014222000839. CIK count: 2. Total Time: 2.51 mins
Done w/ CHEVRON CORP (CIK: 0000093410), Accession #: 000009341022000019. CIK count: 2. Total Time: 2.53 mins
Done w/ CHEVRON CORP (CIK: 0000093410), Accession #: 000009341022000016. CIK count: 2. Total Time: 2.54 mins
Done w/ CHEVRON CORP (CIK: 0000093410), Accession #: 000009341022000009. CIK count: 2. Total Time: 2.56 mins
Done w/ CHEVRON COR

Done w/ CONOCOPHILLIPS (CIK: 0001163165), Accession #: 000116316524000010. CIK count: 5. Total Time: 3.67 mins
Done w/ CONOCOPHILLIPS (CIK: 0001163165), Accession #: 000110465924024352. CIK count: 5. Total Time: 3.68 mins
Done w/ CONOCOPHILLIPS (CIK: 0001163165), Accession #: 000116316524000004. CIK count: 5. Total Time: 3.70 mins
Done w/ CONOCOPHILLIPS (CIK: 0001163165), Accession #: 000116316523000031. CIK count: 5. Total Time: 3.72 mins
Done w/ CONOCOPHILLIPS (CIK: 0001163165), Accession #: 000116316523000029. CIK count: 5. Total Time: 3.73 mins
Done w/ CONOCOPHILLIPS (CIK: 0001163165), Accession #: 000110465923093074. CIK count: 5. Total Time: 3.75 mins
Done w/ CONOCOPHILLIPS (CIK: 0001163165), Accession #: 000116316523000023. CIK count: 5. Total Time: 3.77 mins
Done w/ CONOCOPHILLIPS (CIK: 0001163165), Accession #: 000115752323001246. CIK count: 5. Total Time: 3.78 mins
Done w/ CONOCOPHILLIPS (CIK: 0001163165), Accession #: 000116316523000017. CIK count: 5. Total Time: 3.80 mins
D

Done w/ CONOCOPHILLIPS (CIK: 0001163165), Accession #: 000115752319000952. CIK count: 5. Total Time: 4.80 mins
Done w/ CONOCOPHILLIPS (CIK: 0001163165), Accession #: 000119312519043841. CIK count: 5. Total Time: 4.81 mins
Done w/ CONOCOPHILLIPS (CIK: 0001163165), Accession #: 000115752319000196. CIK count: 5. Total Time: 4.83 mins
Done w/ BP PLC (CIK: 0000313807), Accession #: 000031380723000020. CIK count: 6. Total Time: 4.88 mins
Done w/ BP PLC (CIK: 0000313807), Accession #: 000031380722000025. CIK count: 6. Total Time: 4.90 mins
Done w/ BP PLC (CIK: 0000313807), Accession #: 000031380721000009. CIK count: 6. Total Time: 4.91 mins
Done w/ BP PLC (CIK: 0000313807), Accession #: 000031380720000008. CIK count: 6. Total Time: 4.93 mins
Done w/ BP PLC (CIK: 0000313807), Accession #: 000114036119010687. CIK count: 6. Total Time: 4.95 mins
No 10-K, 10-Q, 8-K, 11-K filings found for EQUINOR ASA (CIK 0001140625)
Done w/ EOG RESOURCES INC (CIK: 0000821189), Accession #: 000082118924000023. CI

...

Done w/ VALERO ENERGY CORP/TX (CIK: 0001035002), Accession #: 000103500224000004. CIK count: 12. Total Time: 11.49 mins
Done w/ VALERO ENERGY CORP/TX (CIK: 0001035002), Accession #: 000119312523276044. CIK count: 12. Total Time: 11.51 mins
Done w/ VALERO ENERGY CORP/TX (CIK: 0001035002), Accession #: 000103500223000110. CIK count: 12. Total Time: 11.52 mins
Done w/ VALERO ENERGY CORP/TX (CIK: 0001035002), Accession #: 000103500223000108. CIK count: 12. Total Time: 11.54 mins
Done w/ VALERO ENERGY CORP/TX (CIK: 0001035002), Accession #: 000119312523236169. CIK count: 12. Total Time: 11.56 mins
Done w/ VALERO ENERGY CORP/TX (CIK: 0001035002), Accession #: 000103500223000103. CIK count: 12. Total Time: 11.57 mins
Done w/ VALERO ENERGY CORP/TX (CIK: 0001035002), Accession #: 000103500223000101. CIK count: 12. Total Time: 11.59 mins
Done w/ VALERO ENERGY CORP/TX (CIK: 0001035002), Accession #: 000119312523191449. CIK count: 12. Total Time: 11.61 mins
Done w/ VALERO ENERGY CORP/TX (CIK: 0001

Done w/ VALERO ENERGY CORP/TX (CIK: 0001035002), Accession #: 000119312520059123. CIK count: 12. Total Time: 12.56 mins
Done w/ VALERO ENERGY CORP/TX (CIK: 0001035002), Accession #: 000119312520057746. CIK count: 12. Total Time: 12.58 mins
Done w/ VALERO ENERGY CORP/TX (CIK: 0001035002), Accession #: 000103500220000007. CIK count: 12. Total Time: 12.60 mins
Done w/ VALERO ENERGY CORP/TX (CIK: 0001035002), Accession #: 000103500220000004. CIK count: 12. Total Time: 12.61 mins
Done w/ VALERO ENERGY CORP/TX (CIK: 0001035002), Accession #: 000119312520014910. CIK count: 12. Total Time: 12.63 mins
Done w/ VALERO ENERGY CORP/TX (CIK: 0001035002), Accession #: 000119312519320890. CIK count: 12. Total Time: 12.64 mins
Done w/ VALERO ENERGY CORP/TX (CIK: 0001035002), Accession #: 000103500219000047. CIK count: 12. Total Time: 12.65 mins
Done w/ VALERO ENERGY CORP/TX (CIK: 0001035002), Accession #: 000103500219000043. CIK count: 12. Total Time: 12.67 mins
Done w/ VALERO ENERGY CORP/TX (CIK: 0001

Done w/ HESS CORP (CIK: 0000004447), Accession #: 000119312520285578. CIK count: 13. Total Time: 13.74 mins
Done w/ HESS CORP (CIK: 0000004447), Accession #: 000162828020014914. CIK count: 13. Total Time: 13.75 mins
Done w/ HESS CORP (CIK: 0000004447), Accession #: 000162828020011992. CIK count: 13. Total Time: 13.77 mins
Done w/ HESS CORP (CIK: 0000004447), Accession #: 000162828020010757. CIK count: 13. Total Time: 13.79 mins
Done w/ HESS CORP (CIK: 0000004447), Accession #: 000156459020029360. CIK count: 13. Total Time: 13.80 mins
Done w/ HESS CORP (CIK: 0000004447), Accession #: 000119312520162246. CIK count: 13. Total Time: 13.82 mins
Done w/ HESS CORP (CIK: 0000004447), Accession #: 000156459020022922. CIK count: 13. Total Time: 13.84 mins
Done w/ HESS CORP (CIK: 0000004447), Accession #: 000156459020022366. CIK count: 13. Total Time: 13.85 mins
Done w/ HESS CORP (CIK: 0000004447), Accession #: 000119312520076111. CIK count: 13. Total Time: 13.87 mins
Done w/ HESS CORP (CIK: 0000

Done w/ DEVON ENERGY CORP/DE (CIK: 0001090012), Accession #: 000119312520324314. CIK count: 14. Total Time: 14.96 mins
Done w/ DEVON ENERGY CORP/DE (CIK: 0001090012), Accession #: 000156459020049203. CIK count: 14. Total Time: 14.98 mins
Done w/ DEVON ENERGY CORP/DE (CIK: 0001090012), Accession #: 000119312520281183. CIK count: 14. Total Time: 14.99 mins
Done w/ DEVON ENERGY CORP/DE (CIK: 0001090012), Accession #: 000119312520262518. CIK count: 14. Total Time: 15.01 mins
Done w/ DEVON ENERGY CORP/DE (CIK: 0001090012), Accession #: 000119312520255422. CIK count: 14. Total Time: 15.02 mins
Done w/ DEVON ENERGY CORP/DE (CIK: 0001090012), Accession #: 000156459020036568. CIK count: 14. Total Time: 15.04 mins
Done w/ DEVON ENERGY CORP/DE (CIK: 0001090012), Accession #: 000119312520209396. CIK count: 14. Total Time: 15.05 mins
Done w/ DEVON ENERGY CORP/DE (CIK: 0001090012), Accession #: 000119312520161458. CIK count: 14. Total Time: 15.07 mins
Done w/ DEVON ENERGY CORP/DE (CIK: 0001090012), 

Done w/ MARATHON OIL CORP (CIK: 0000101778), Accession #: 000010177821000086. CIK count: 15. Total Time: 16.10 mins
Done w/ MARATHON OIL CORP (CIK: 0000101778), Accession #: 000010177821000083. CIK count: 15. Total Time: 16.12 mins
Done w/ MARATHON OIL CORP (CIK: 0000101778), Accession #: 000010177821000080. CIK count: 15. Total Time: 16.14 mins
Done w/ MARATHON OIL CORP (CIK: 0000101778), Accession #: 000010177821000075. CIK count: 15. Total Time: 16.15 mins
Done w/ MARATHON OIL CORP (CIK: 0000101778), Accession #: 000010177821000061. CIK count: 15. Total Time: 16.16 mins
Done w/ MARATHON OIL CORP (CIK: 0000101778), Accession #: 000010177821000018. CIK count: 15. Total Time: 16.18 mins
Done w/ MARATHON OIL CORP (CIK: 0000101778), Accession #: 000010177821000013. CIK count: 15. Total Time: 16.20 mins
Done w/ MARATHON OIL CORP (CIK: 0000101778), Accession #: 000010177821000003. CIK count: 15. Total Time: 16.21 mins
Done w/ MARATHON OIL CORP (CIK: 0000101778), Accession #: 00001017782000